In [7]:
import csv
import re
import pandas as pd

In [8]:
def parse_pair(text):
    text = re.sub(r"\@", "", text)
    pairs = text.split("\t")
    pairs_list = list()
    for pair in pairs:
        pairs_list.append(tuple(pair.split(",")))
    return pairs_list

In [9]:
pairs_list = list()
label_list = list()
with open('train_return_pair.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        label_list.append(row[1])
        pairs_list.append(parse_pair(row[2]))

In [10]:
pair_set = set()
for pairs in pairs_list:
    for pair in pairs:
        if len(pair) == 3:
            pair_set.add((pair[0], pair[1]))

In [11]:
pair_list = list(pair_set)
pair_dict = {}
for i in range(len(pair_list)):
    pair_dict[pair_list[i]] = i

In [64]:
import numpy as np
train_arr = np.zeros((len(pairs_list), len(pair_list)))
train_arr.fill(-100)
for i in range(len(pairs_list)):
    pairs = pairs_list[i]
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = float(pair[2])
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            train_arr[i, pair_dict[pair_label]] = value

In [65]:
'''
train_df = pd.DataFrame(columns=pair_list)
columns_num = len(train_df.columns)
for i in range(len(pairs_list)):
    pairs = pairs_list[i]
    row = [-1] * columns_num
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = pair[2]
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            row[pair_dict[pair_label]] = value
    new_df =  pd.DataFrame(columns=pair_list)
    new_df.loc[0] = row
    train_df = train_df.append(new_df)
    print(len(train_df))
    
'''

'\ntrain_df = pd.DataFrame(columns=pair_list)\ncolumns_num = len(train_df.columns)\nfor i in range(len(pairs_list)):\n    pairs = pairs_list[i]\n    row = [-1] * columns_num\n    for pair in pairs:\n        if len(pair) != 3:\n            continue\n        value = pair[2]\n        pair_label = (pair[0], pair[1])\n        if pair_label in pair_dict:\n            row[pair_dict[pair_label]] = value\n    new_df =  pd.DataFrame(columns=pair_list)\n    new_df.loc[0] = row\n    train_df = train_df.append(new_df)\n    print(len(train_df))\n    \n'

In [67]:
pairs_test_list = list()
label_test_list = list()
with open('test_return_pair.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        label_test_list.append(row[1])
        pairs_test_list.append(parse_pair(row[2]))

In [68]:
test_arr = np.zeros((len(pairs_test_list), len(pair_list)))
test_arr.fill(-100)
for i in range(len(pairs_test_list)):
    pairs = pairs_test_list[i]
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = float(pair[2])
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            test_arr[i, pair_dict[pair_label]] = value
'''


test_df = pd.DataFrame(columns=pair_list)
columns_num = len(test_df.columns)
for i in range(len(pairs_test_list)):
    pairs = pairs_test_list[i]
    row = [-1] * columns_num
    for pair in pairs:
        if len(pair) != 3:
            continue
        value = float(pair[2])
        pair_label = (pair[0], pair[1])
        if pair_label in pair_dict:
            row[pair_dict[pair_label]] = value
    test_df.loc[i] = row
'''

'\n\n\ntest_df = pd.DataFrame(columns=pair_list)\ncolumns_num = len(test_df.columns)\nfor i in range(len(pairs_test_list)):\n    pairs = pairs_test_list[i]\n    row = [-1] * columns_num\n    for pair in pairs:\n        if len(pair) != 3:\n            continue\n        value = float(pair[2])\n        pair_label = (pair[0], pair[1])\n        if pair_label in pair_dict:\n            row[pair_dict[pair_label]] = value\n    test_df.loc[i] = row\n'

### Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_arr, label_list)
y_pred = clf.predict(test_arr)
from sklearn.metrics import accuracy_score
accuracy_score(label_test_list, y_pred)

0.55079271873165003

### Decision Tree

In [74]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(train_arr, label_list).predict(test_arr)
accuracy_score(label_test_list, y_pred)

0.53024075161479745

### Naive Bayes

In [75]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(train_arr, label_list).predict(test_arr)
accuracy_score(label_test_list, y_pred)

0.50264239577216674